In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from pycaret.classification import save_model, setup
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import (classification_report)
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
# Caminhos para as pastas de áudio
pasta_ruido = r'../audios/audios_treinamento/classe_ruido'
pasta_barco = r'../audios/audios_treinamento/classe_barco'

# Função para extrair features dos áudios
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    features = {
        'chroma_stft': librosa.feature.chroma_stft(y=y, sr=sr).mean(),
        'chroma_cqt': librosa.feature.chroma_cqt(y=y, sr=sr).mean(),
        'chroma_cens': librosa.feature.chroma_cens(y=y, sr=sr).mean(),
        'rmse': librosa.feature.rms(y=y).mean(),
        'spectral_centroid': librosa.feature.spectral_centroid(y=y, sr=sr).mean(),
        'spectral_bandwidth': librosa.feature.spectral_bandwidth(y=y, sr=sr).mean(),
        'spectral_contrast': librosa.feature.spectral_contrast(y=y, sr=sr).mean(),
        'spectral_flatness': librosa.feature.spectral_flatness(y=y).mean(),
        'spectral_rolloff': librosa.feature.spectral_rolloff(y=y, sr=sr).mean(),
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(y).mean(),
        'mfcc1': librosa.feature.mfcc(y=y, sr=sr)[0].mean(),
        'mfcc2': librosa.feature.mfcc(y=y, sr=sr)[1].mean(),
        'mfcc3': librosa.feature.mfcc(y=y, sr=sr)[2].mean(),
        # 'mfcc4': librosa.feature.mfcc(y=y, sr=sr)[3].mean(),
        # 'mfcc5': librosa.feature.mfcc(y=y, sr=sr)[4].mean(),
        # 'mfcc6': librosa.feature.mfcc(y=y, sr=sr)[5].mean(),
        # 'mfcc7': librosa.feature.mfcc(y=y, sr=sr)[6].mean(),
        # 'mfcc8': librosa.feature.mfcc(y=y, sr=sr)[7].mean(),
        # 'mfcc9': librosa.feature.mfcc(y=y, sr=sr)[8].mean(),
        # 'mfcc10': librosa.feature.mfcc(y=y, sr=sr)[9].mean(),
        # 'mfcc11': librosa.feature.mfcc(y=y, sr=sr)[10].mean(),
        # 'mfcc12': librosa.feature.mfcc(y=y, sr=sr)[11].mean(),
        # 'mfcc13': librosa.feature.mfcc(y=y, sr=sr)[12].mean(),
        #'tempo': librosa.beat.tempo(y=y, sr=sr)[0]
    }
    return features

# Função para processar as pastas e adicionar os dados ao DataFrame
def verifica_pastas(folder_path, label):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith('.wav'):
            features = extract_features(file_path)
            features['label'] = label
            data.append(features)
    return data

# Processar as duas pastas sendo cada uma uma classe
dados_ruido = verifica_pastas(pasta_ruido, 1)
dados_barco = verifica_pastas(pasta_barco, 2)

# Combinar os dados e criar um DataFrame
data = dados_ruido + dados_barco
df = pd.DataFrame(data)

# Salvar em um arquivo CSV
df.to_csv("CSV\\features_audios_classificados.csv", index=False)

In [ ]:
dados = pd.read_csv("CSV\\features_audios_classificados.csv")
X = dados.drop(columns=['label'])
y = dados['label']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Treinando o modelo LDA
lda_model = LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)
lda_model.fit(X_train, y_train)

# Fazer previsões e avaliar o modelo
y_pred = lda_model.predict(X_test)
print("Relatório de Classificação para LDA:")
print(classification_report(y_test, y_pred))

clf1 = setup(data=dados, target='label', session_id=123)
save_model(lda_model, 'Modelo_LDA')

In [ ]:


scores = cross_val_score(lda_model, X, y, cv=5)
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {np.mean(scores)}")